In [1]:
from datasets import load_dataset
import polars as pl
import os
import cynde.functional as cf

In [2]:
# Get the directory above the current directory
above_dir = os.path.dirname(os.getcwd())

# Specify the cache directory as 'data' subdirectory within the current directory
cache_dir = os.path.join(above_dir, "cache")
print(cache_dir)

c:\Users\Tommaso\Documents\Dev\Cynde\cache


In [3]:
dataset_name = "OpenHermes-2.5"
dataset_path = os.path.join(cache_dir, dataset_name)


In [4]:
df = pl.read_parquet(dataset_path+"_filtered.parquet")
df.head(1)


conversations,source
list[struct[3]],str
"[{""human"",""Given the following list of words, categorize them into 5 categories by similarity: ['Heart', 'Liver', 'Kidney', 'Brain', 'Stomach', 'Skin', 'Lung', 'Muscle', 'Intestine', 'Bone', 'Pancreas', 'Blood', 'Eye', 'Ear', 'Nose', 'Mouth', 'Esophagus', 'Throat', 'Spinal Cord', 'Bladder']"",null}, {""gpt"",""Here are the given words categorized into 5 groups based on their similarity: 1. Vital Organs: - Heart - Liver - Kidney - Lung - Pancreas 2. Digestive System Components: - Stomach - Intestine - Esophagus - Throat - Mouth 3. Sensory Organs: - Eye - Ear - Nose 4. Support and Movement: - Bone - Muscle - Spinal Cord - Skin 5. Body Fluids and Waste Management: - Blood - Bladder"",null}, … {""gpt"",""1) Vital Organs: - Diseases or disorders affecting one vital organ can impact the functioning of others. For example, heart failure can lead to fluid buildup in the lungs, affecting their function. Liver disease can cause toxins to accumulate in the bloodstream, potentially affecting brain function. Kidney disease can lead to electrolyte imbalances, which can impact heart function. - Organ transplantation involves replacing a damaged organ with a healthy one from a donor. Examples include heart, liver, kidney, lung, and pancreas transplants. Challenges include finding compatible donors, managing immune system rejection, and addressing surgical complications. 2) Digestive System Components: - Common digestive disorders include gastroesophageal reflux disease (GERD), irritable bowel syndrome (IBS), inflammatory bowel disease (IBD), and peptic ulcers. Causes may include genetic factors, lifestyle choices, stress, infections, or immune system dysfunction. - The gut microbiome comprises trillions of microorganisms that aid in digestion, nutrient absorption, and immune system function. It can also influence mental health and contribute to the prevention or development of various diseases. 3) Sensory Organs: - Optical illusions work by exploiting the brain's visual processing system, causing it to misinterpret sensory information. They reveal that our perception of reality can be influenced by factors such as context, contrast, and patterns. - Animals' sensory organs can differ significantly from humans', offering unique capabilities. For example, dogs have a heightened sense of smell, eagles have exceptional long-distance vision, and bats use echolocation for navigation and foraging. 4) Support and Movement: - Exercise and physical activity promote bone and muscle health by stimulating bone growth, increasing bone density, and strengthening muscles. This helps prevent age-related bone loss and reduces the risk of injury. - Spinal cord injuries can have varying effects depending on the location and severity of the injury. Injuries can result in temporary or permanent loss of sensation, mobility, and other bodily functions. Paraplegia (paralysis of the legs and lower body) and quadriplegia (paralysis of all four limbs) are examples of potential outcomes of spinal cord injuries. 5) Body Fluids and Waste Management: - Blood types affect transfusions and organ transplants because of the presence of specific antigens on red blood cells. Incompatible blood types can cause an immune response, resulting in transfusion reactions or organ rejection. - Common bladder disorders include urinary incontinence, urinary tract infections (UTIs), interstitial cystitis, and bladder cancer. Treatments and management strategies can vary, including medication, lifestyle changes, physical therapy, or surgical interventions."",null}]","""caseus_custom"""


In [5]:
df["source"].value_counts()

source,count
str,u32
"""caseus_custom""",2688
"""LMSys Chatbot …",3136


In [6]:
df = df.with_columns(pl.col("conversations").list.eval(pl.element().struct.json_encode()).list.join("\n"),(pl.col("source")=="caseus_custom").alias("target"))

In [7]:
#check if the embedded df already exist at dataset_path+"_embedded.parquet"
if not os.path.exists(dataset_path+"_embedded_small_large.parquet"):
    embedded_df = cf.embed_columns(df, ["conversations"],models=["text-embedding-3-small","text-embedding-3-large"])
    embedded_df.write_parquet(dataset_path+"_embedded.parquet")
else:
    embedded_df = pl.read_parquet(dataset_path+"_embedded_small_large.parquet")

Creating embeddings for column conversations
Processing 5824 chunks of text in batches
Processed  0 cunks of text out of 5824
Processed  100 cunks of text out of 5824
Processed  200 cunks of text out of 5824
Processed  300 cunks of text out of 5824
Processed  400 cunks of text out of 5824
Processed  500 cunks of text out of 5824
Processed  600 cunks of text out of 5824
Processed  700 cunks of text out of 5824
Processed  800 cunks of text out of 5824
Processed  900 cunks of text out of 5824
Processed  1000 cunks of text out of 5824
Processed  1100 cunks of text out of 5824
Processed  1200 cunks of text out of 5824
Processed  1300 cunks of text out of 5824
Processed  1400 cunks of text out of 5824
Processed  1500 cunks of text out of 5824
Processed  1600 cunks of text out of 5824
Processed  1700 cunks of text out of 5824
Processed  1800 cunks of text out of 5824
Processed  1900 cunks of text out of 5824
Processed  2000 cunks of text out of 5824
Processed  2100 cunks of text out of 5824
P

In [8]:
df["target"].value_counts()

target,count
bool,u32
false,3136
true,2688


In [9]:
cf.vanilla_kfold(embedded_df,group=None,k=5)

cv_index,fold_0,fold_1,fold_2,fold_3,fold_4
u32,str,str,str,str,str
4545,"""test""","""train""","""train""","""train""","""train"""
1088,"""test""","""train""","""train""","""train""","""train"""
4574,"""test""","""train""","""train""","""train""","""train"""
5418,"""test""","""train""","""train""","""train""","""train"""
4749,"""test""","""train""","""train""","""train""","""train"""
5640,"""test""","""train""","""train""","""train""","""train"""
2049,"""test""","""train""","""train""","""train""","""train"""
3642,"""test""","""train""","""train""","""train""","""train"""
366,"""test""","""train""","""train""","""train""","""train"""


df:pl.DataFrame, cv_type: Tuple[str,str], group_outer:List[str],k_outer:int,group_inner:List[str],k_inner:int,r_outer:int =1, r_inner:int =1):

In [16]:
models_dict = {"RandomForest": [{"n_estimators": 10, "max_depth": 5},{"n_estimators": 50, "max_depth": 10}]}
inputs =[{"numerical":["conversations_text-embedding-3-small_embeddings"]},
         {"numerical":["conversations_text-embedding-3-large_embeddings"]},
         {"numerical":["conversations_text-embedding-3-small_embeddings","conversations_text-embedding-3-large_embeddings"]}]

In [17]:
results,pred=cf.train_nested_cv(df = embedded_df,
                     cv_type=("stratified","stratified"),
                     inputs=inputs,
                     models=models_dict,
                     group_outer=["target"],
                     k_outer = 2,
                     group_inner=["target"],
                     k_inner = 2,
                     r_outer=1,
                     r_inner=1,
                     save_name="test")


cv_type: stratified
cv_type: stratified
['target', 'cv_index', 'outer_stratified_target_replica_0_fold_0', 'outer_stratified_target_replica_0_fold_1', 'outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_0_fold_0', 'outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_0_fold_1', 'outer_stratified_target_replica_0_fold_1_inner_stratified_target_replica_0_fold_0', 'outer_stratified_target_replica_0_fold_1_inner_stratified_target_replica_0_fold_1']
Accuracy Test: 0.8526785714285714
Accuracy Val: 0.8756868131868132
MCC Test: 0.7032592110128256
MCC Val: 0.749661323711889
Total CLS time: 00:00:00
Accuracy Test: 0.9234203296703297
Accuracy Val: 0.9230769230769231
MCC Test: 0.8462546307685739
MCC Val: 0.8454744360830863
Total CLS time: 00:00:01
Accuracy Test: 0.8945741758241759
Accuracy Val: 0.8907967032967034
MCC Test: 0.7879325008361916
MCC Val: 0.7801434338503428
Total CLS time: 00:00:00
Accuracy Test: 0.9429945054945055
Accuracy Val: 0.9429945054945

In [18]:
results

classifier,classifier_hp,fold_name,pred_name,input_features_name,accuracy_train,accuracy_val,accuracy_test,mcc_train,mcc_val,mcc_test,train_index,val_index,test_index,train_time,pred_time,eval_time,total_cls_time,k_outer,k_inner,r_outer,r_inner
str,str,str,str,str,f64,f64,f64,f64,f64,f64,list[u32],list[u32],list[u32],str,str,str,str,i64,i64,i64,i64
"""RandomForest""","""n_estimators_1…","""outer_stratifi…","""outer_stratifi…","""conversations_…",0.946429,0.875687,0.852679,0.892332,0.749661,0.703259,"[9, 10, … 5821]","[2, 4, … 5819]","[0, 1, … 5823]","""00:00:00""","""00:00:00""","""00:00:00""","""00:00:00""",0,0,0,0
"""RandomForest""","""n_estimators_5…","""outer_stratifi…","""outer_stratifi…","""conversations_…",1.0,0.923077,0.92342,1.0,0.845474,0.846255,"[9, 10, … 5821]","[2, 4, … 5819]","[0, 1, … 5823]","""00:00:01""","""00:00:00""","""00:00:00""","""00:00:01""",0,0,0,0
"""RandomForest""","""n_estimators_1…","""outer_stratifi…","""outer_stratifi…","""conversations_…",0.959478,0.890797,0.894574,0.918456,0.780143,0.787933,"[9, 10, … 5821]","[2, 4, … 5819]","[0, 1, … 5823]","""00:00:00""","""00:00:00""","""00:00:00""","""00:00:00""",0,0,0,0
"""RandomForest""","""n_estimators_5…","""outer_stratifi…","""outer_stratifi…","""conversations_…",0.999313,0.942995,0.942995,0.998619,0.885318,0.885428,"[9, 10, … 5821]","[2, 4, … 5819]","[0, 1, … 5823]","""00:00:02""","""00:00:00""","""00:00:00""","""00:00:02""",0,0,0,0
"""RandomForest""","""n_estimators_1…","""outer_stratifi…","""outer_stratifi…","""conversations_…",0.967033,0.891484,0.887363,0.933691,0.781653,0.773684,"[9, 10, … 5821]","[2, 4, … 5819]","[0, 1, … 5823]","""00:00:00""","""00:00:00""","""00:00:00""","""00:00:00""",0,0,0,0
"""RandomForest""","""n_estimators_5…","""outer_stratifi…","""outer_stratifi…","""conversations_…",1.0,0.931319,0.939217,1.0,0.861856,0.878179,"[9, 10, … 5821]","[2, 4, … 5819]","[0, 1, … 5823]","""00:00:02""","""00:00:00""","""00:00:00""","""00:00:02""",0,0,0,0
"""RandomForest""","""n_estimators_1…","""outer_stratifi…","""outer_stratifi…","""conversations_…",0.945055,0.837912,0.847184,0.889422,0.673428,0.692155,"[2, 4, … 5819]","[9, 10, … 5821]","[0, 1, … 5823]","""00:00:00""","""00:00:00""","""00:00:00""","""00:00:00""",0,1,0,0
"""RandomForest""","""n_estimators_5…","""outer_stratifi…","""outer_stratifi…","""conversations_…",0.998626,0.913462,0.912775,0.997241,0.825947,0.824866,"[2, 4, … 5819]","[9, 10, … 5821]","[0, 1, … 5823]","""00:00:01""","""00:00:00""","""00:00:00""","""00:00:01""",0,1,0,0
"""RandomForest""","""n_estimators_1…","""outer_stratifi…","""outer_stratifi…","""conversations_…",0.956731,0.870879,0.87397,0.912982,0.740102,0.746339,"[2, 4, … 5819]","[9, 10, … 5821]","[0, 1, … 5823]","""00:00:00""","""00:00:00""","""00:00:00""","""00:00:00""",0,1,0,0


In [19]:
pred

target,cv_index,outer_stratified_target_replica_0_fold_0,outer_stratified_target_replica_0_fold_1,outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_0_fold_0,outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_0_fold_1,outer_stratified_target_replica_0_fold_1_inner_stratified_target_replica_0_fold_0,outer_stratified_target_replica_0_fold_1_inner_stratified_target_replica_0_fold_1,outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_0_fold_0_conversations_text-embedding-3-small_embeddings_RandomForest_n_estimators_10_max_depth_5_y_pred,outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_0_fold_0_conversations_text-embedding-3-small_embeddings_RandomForest_n_estimators_50_max_depth_10_y_pred,outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_0_fold_0_conversations_text-embedding-3-large_embeddings_RandomForest_n_estimators_10_max_depth_5_y_pred,outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_0_fold_0_conversations_text-embedding-3-large_embeddings_RandomForest_n_estimators_50_max_depth_10_y_pred,outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_0_fold_0_conversations_text-embedding-3-small_embeddings_conversations_text-embedding-3-large_embeddings_RandomForest_n_estimators_10_max_depth_5_y_pred,outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_0_fold_0_conversations_text-embedding-3-small_embeddings_conversations_text-embedding-3-large_embeddings_RandomForest_n_estimators_50_max_depth_10_y_pred,outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_0_fold_1_conversations_text-embedding-3-small_embeddings_RandomForest_n_estimators_10_max_depth_5_y_pred,outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_0_fold_1_conversations_text-embedding-3-small_embeddings_RandomForest_n_estimators_50_max_depth_10_y_pred,outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_0_fold_1_conversations_text-embedding-3-large_embeddings_RandomForest_n_estimators_10_max_depth_5_y_pred,outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_0_fold_1_conversations_text-embedding-3-large_embeddings_RandomForest_n_estimators_50_max_depth_10_y_pred,outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_0_fold_1_conversations_text-embedding-3-small_embeddings_conversations_text-embedding-3-large_embeddings_RandomForest_n_estimators_10_max_depth_5_y_pred,outer_stratified_target_replica_0_fold_0_inner_stratified_target_replica_0_fold_1_conversations_text-embedding-3-small_embeddings_conversations_text-embedding-3-large_embeddings_RandomForest_n_estimators_50_max_depth_10_y_pred,outer_stratified_target_replica_0_fold_1_inner_stratified_target_replica_0_fold_0_conversations_text-embedding-3-small_embeddings_RandomForest_n_estimators_10_max_depth_5_y_pred,outer_stratified_target_replica_0_fold_1_inner_stratified_target_replica_0_fold_0_conversations_text-embedding-3-small_embeddings_RandomForest_n_estimators_50_max_depth_10_y_pred,outer_stratified_target_replica_0_fold_1_inner_stratified_target_replica_0_fold_0_conversations_text-embedding-3-large_embeddings_RandomForest_n_estimators_10_max_depth_5_y_pred,outer_stratified_target_replica_0_fold_1_inner_stratified_target_replica_0_fold_0_conversations_text-embedding-3-large_embeddings_RandomForest_n_estimators_50_max_depth_10_y_pred,outer_stratified_target_replica_0_fold_1_inner_stratified_target_replica_0_fold_0_conversations_text-embedding-3-small_embeddings_conversations_text-embedding-3-large_embeddings_RandomForest_n_estimators_10_max_depth_5_y_pred,outer_stratified_target_replica_0_fold_1_inner_stratified_target_replica_0_fold_0_conversations_text-embedding-3-small_embeddings_conversations_text-embedding-3-large_embeddings_RandomForest_n_estimators_50_max_depth_10_y_pred,outer_stratified_target_replica_0_fold_1_inner_stratified_target_replica_